<img src = 'https://upload.wikimedia.org/wikipedia/commons/2/26/World_Health_Organization_Logo.svg'
    width = 690px
    height= 665px />

# **WHO Life Expectancy  —  Non-sensitive Linear Model**

by: Team 1 - Scrum

***

## **The Data**

|Column|Description|
|---:|:---|
|Country|Country|
|Region|Region|
|Year|Year|
|Infant_deaths|Number of Infant Deaths per 1000 population|
|Under_five_deaths|Number of under-five deaths per 1000 population|
|Adult_mortality|Adult Mortality Rates of both sexes (probability of dying between 15 and 60 years per 1000 population)|
|Alcohol_consumption|Alcohol, recorded per capita (15+) consumption (in litres of pure alcohol)|
|Hepatitis B|Hepatitis B (HepB) immunization coverage among 1-year-olds (%)|
|Measles|Measles - number of reported cases per 1000 population|
|BMI|Average Body Mass Index of entire population|
|Polio|Polio (Pol3) immunization coverage among 1-year-olds (%)|
|Diphtheria|Diphtheria tetanus toxoid and pertussis (DTP3) immunization coverage among 1-year-olds (%)|
|Incidents_HIV|Deaths per 1 000 live births HIV/AIDS (0-4 years)|
|GDP_per_capita|Gross Domestic Product per capita (in USD)|
|Population_mln|Population of the country|
|Thinness_ten_nineteen_years|Prevalence of thinness among children and adolescents for Age 10 to 19 (%)|
|Thinness_five_nine_years|Prevalence of thinness among children for Age 5 to 9(%)|
|Schooling|Number of years of Schooling(years)|
|Economy_status_Developed|Developed status|
|Economy_status_Developing|Developing status|
|Life expectancy|Life Expectancy in age|

## **The Scope**

This notebook explores the Life Expectancy dataset from the World Health Organization.\
It outlines the process for building a robust linear regression model that accurately predicts life expectancy.\
The model incorporates some insights gained from the exploratory data analysis (refer to `WHO - EDA.ipynb` file)

## **Key Questions Answered**

* How can we implement the insights gained from the exploratory data analysis?
* Which features are most influential in predicting life expectancy?
* How well does a linear regression model perform on this dataset?

***

### 1. Dataset Import

In [1]:
# Importing required modules

import pandas as pd    # for general data use & data analysis
import matplotlib.pyplot as plt    # for data visualisation
import seaborn as sns  # for data visualisation
import numpy as np     # for maths

from sklearn.model_selection import train_test_split    # for performing train-test split on the data

# Use statsmodels for both the model and its evaluation
import statsmodels.api as sm    # we'll get the model from
import statsmodels.tools        # we'll get the evaluation metrics from

from sklearn.preprocessing import RobustScaler   # for scaling the data

from statsmodels.stats.outliers_influence import variance_inflation_factor   # for checking VIF

The DataFrame `df` is created by reading the `Life Expectancy Data.csv` file from a local folder.

In [2]:
df = pd.read_csv('Life Expectancy Data.csv')

In [3]:
# Checking out first 5 rows of data
pd.set_option('display.max_columns', None)
df.head()

,Country,Region,Year,Infant_deaths,Under_five_deaths,Adult_mortality,Alcohol_consumption,Hepatitis_B,Measles,BMI,Polio,Diphtheria,Incidents_HIV,GDP_per_capita,Population_mln,Thinness_ten_nineteen_years,Thinness_five_nine_years,Schooling,Economy_status_Developed,Economy_status_Developing,Life_expectancy
0,Turkiye,Middle East,2015,11.1,13.0,105.8240,1.32,97,65,27.8,97,97,0.08,11006,78.53,4.9,4.8,7.8,0,1,76.5
1,Spain,European Union,2015,2.7,3.3,57.9025,10.35,97,94,26.0,97,97,0.09,25742,46.44,0.6,0.5,9.7,1,0,82.8
2,India,Asia,2007,51.5,67.9,201.0765,1.57,60,35,21.2,67,64,0.13,1076,1183.21,27.1,28.0,5.0,0,1,65.4
3,Guyana,South America,2006,32.8,40.5,222.1965,5.68,93,74,25.3,92,93,0.79,4146,0.75,5.7,5.5,7.9,0,1,67.0
4,Israel,Middle East,2012,3.4,4.3,57.9510,2.89,97,89,27.0,94,94,0.08,33995,7.91,1.2,1.1,12.8,1,0,81.7


Inspecting the dataset we can observe that there is a total of `2864` observations and `21` columns. Only 2 columns have `object` datatype: `Country` and `Region`. The rest of the columns are either `int64` or `float64`.

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2864 entries, 0 to 2863
Data columns (total 21 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Country                      2864 non-null   object 
 1   Region                       2864 non-null   object 
 2   Year                         2864 non-null   int64  
 3   Infant_deaths                2864 non-null   float64
 4   Under_five_deaths            2864 non-null   float64
 5   Adult_mortality              2864 non-null   float64
 6   Alcohol_consumption          2864 non-null   float64
 7   Hepatitis_B                  2864 non-null   int64  
 8   Measles                      2864 non-null   int64  
 9   BMI                          2864 non-null   float64
 10  Polio                        2864 non-null   int64  
 11  Diphtheria                   2864 non-null   int64  
 12  Incidents_HIV                2864 non-null   float64
 13  GDP_per_capita    

### 2. Train-Test Split

Now that we have our data loaded let's perform a train-test split. The train set will be used to fit the model and the test set to evaluate.\
Note that the independent variable or target is `Life_expectancy`.

Let's first make a list of all of our features, and then remove `Life_expectancy` from that list, since we won't include it in `X`.

In [5]:
sensitive_cols = ["Infant_deaths", "Under_five_deaths", "Hepatitis_B","Polio",
                  "Diphtheria","Incidents_HIV","Thinness_ten_nineteen_years",
                  "Thinness_five_nine_years","Schooling","Economy_status_Developed",
                  "Economy_status_Developing"]

feature_cols = list(df.columns) # the columns that we'll se as features

for cols in sensitive_cols:
    feature_cols.remove(cols)
feature_cols.remove('Life_expectancy') # 'Life_expectancy' removed because it's the target

Using the list of features we can now create `X` with all the features and `y` with the target only.

In [6]:
X = df[feature_cols] # our features dataframe --> It will become: X_train and X_test
y = df['Life_expectancy'] # our target --> It will become: y_train and y_test

Let's check out `X` and `y`.

In [7]:
X.head(3)

,Country,Region,Year,Adult_mortality,Alcohol_consumption,Measles,BMI,GDP_per_capita,Population_mln
0,Turkiye,Middle East,2015,105.8240,1.32,65,27.8,11006,78.53
1,Spain,European Union,2015,57.9025,10.35,94,26.0,25742,46.44
2,India,Asia,2007,201.0765,1.57,35,21.2,1076,1183.21


In [8]:
y.head(3)

,Life_expectancy
0,76.5
1,82.8
2,65.4


Next, we will perform a train-test split, allocating `80%` of the data for training and `20%` for testing.\
Using `train_test_split` we'll return 4 items: X_train, X_test, y_test and y_train.

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, # the features
                                                    y, # the target
                                                    test_size = 0.2, # 20% test size
                                                    random_state = 1) # setting a random seed for reproducibility of results

Let's check out `X_train` and `y_train`.

In [10]:
X_train.head(3)

,Country,Region,Year,Adult_mortality,Alcohol_consumption,Measles,BMI,GDP_per_capita,Population_mln
2291,United Arab Emirates,Middle East,2001,95.5935,1.62,92,27.2,58422,3.30
1188,Spain,European Union,2008,70.1140,10.24,94,26.2,27026,45.95
772,Sao Tome and Principe,Central America and Caribbean,2002,217.4790,5.93,71,23.7,1087,0.15


In [11]:
y_train.head(3)

,Life_expectancy
2291,74.5
1188,81.2
772,62.7


Next we perform sanity checks to confirm that:
* The indexes of X_train and with y_train coincide
* The indexes of X_test and with y_test coincide

The `all()` function returns `True` if ALL of the elements are True, False otherwise.

In [12]:
print(all(X_train.index == y_train.index))
print(all(X_test.index == y_test.index))

True
True


In [13]:
print(X_train.shape[0] == len(y_train))
print(X_test.shape[0] == len(y_test))

True
True


All sanity checks passed.

Please note that in the exploratory data analysis notebook `WHO - EDA.ipynb`, the dataset underwent thorough inspection, including data quality checks, data types and duplicate and missing values detection. Therefore, these steps are skipped in this notebook.

### 3. Feature Engineering

Let's revisit the key takeaways from the exploratory data analysis and the actions that will be carried forward into the feature engineering phase:

* _Columns with object data type such as `Country` and `Region` can be one-hot encoded, as they don't contain ordinal data. However, `Country` has 179 unique values and one-hot encoding would result in 179 new features and would lead to dimensionality issues._

  **Action:** The `Region` column will be one-hot encoded. This will create `9` additional binary columns: `Region_Africa`, `Region_European Union`, `Region_Asia`, `Region_Central America and Caribbean`, `Region_Rest of Europe`, `Region_Middle East`, `Region_South America`, `Region_Oceania`, `Region_North America`. The `Country` column will be dropped, as one-hot encoding it and using all its 179 columns would lead to an unstable linear model with an extremely high condition number.

* _The `Economy_status_Developed` and `Economy_status_Developing` columns have redundant information. We can prescind of one of those columns._

  **Action:** The `Economy_status_Developed` column will be removed.

* _There are a number of outliers present in several columns. One approach to mitigating their impact could be to apply a transformation or, alternatively, remove some of them._

  **Action:** all outliers will retained. Data scaling will be performed to mitigate their impact.

* _There are several features that are highly correlated and could potentially be combined when doing feature engineering._

  **Action:** the following features will be combined to address multicollinearity in the model:
  - `Diphtheria` and `Polio`
  - `Under_five_deaths` and `Infant_deaths`
  - `Thinness_ten_nineteen_years` and `Thinness_five_nine_years`


Along with `Country` and `Economy_status_Developed` columns, the `Year` column will also be dropped for following reasons:
* `Year` had a weakly positive correlation with `Life_expectancy`, showing that it doesn't directly influence life expectancy.
* If a model were to learn to predict life expectancy based on the year, it would have issues generalising and would struggle to predict on new data.

Let’s now create our general function for feature engineering to implement the actions outlined above.

In [14]:
def feature_eng(df):

    # Creating a copy of the DataFrame
    df = df.copy()

    # One-hot enconding the 'Region' column and dropping it afterwards
    df = pd.get_dummies(df, columns=['Region'], drop_first=True, prefix='Region',dtype=int)
    # df = pd.get_dummies(df, columns=['Country'], drop_first=True, prefix='Country',dtype=int)

    # Feature enrichment - creating new columns from combining highly correlated features
    df['log_population'] = np.log1p(X['Population_mln'])  # log(1 + x) to avoid log(0)

    # # dropping the columns that have been combined
    df.drop(columns=['Country','Population_mln'], inplace=True)

    # adding a constant
    df = sm.add_constant(df)

    return df  # returning the DatFrame

Let's now apply the `feature_eng()` function to both the training and testing sets, creating new variables named `X_train_fe` and `X_test_fe`.

In [15]:
X_train_fe = feature_eng(X_train)
X_test_fe = feature_eng(X_test)

Next we perform sanity checks to confirm that:
* The indexes of X_train_fe and with y_train coincide
* The indexes of X_test_fe and with y_test coincide

The `all()` function returns `True` if ALL of the elements are True, False otherwise.

In [16]:
print(all(X_train_fe.index == y_train.index))
print(all(X_test_fe.index == y_test.index))

True
True


In [17]:
print(X_train_fe.shape[0] == len(y_train))
print(X_test_fe.shape[0] == len(y_test))

True
True


All sanity checks passed.

Let's check out the first few rows of `X_train_fe`.

In [18]:
pd.set_option('display.max_columns', None)
X_train_fe.head()

,const,Year,Adult_mortality,Alcohol_consumption,Measles,BMI,GDP_per_capita,Region_Asia,Region_Central America and Caribbean,Region_European Union,Region_Middle East,Region_North America,Region_Oceania,Region_Rest of Europe,Region_South America,log_population
2291,1.0,2001,95.5935,1.62,92,27.2,58422,0,0,0,1,0,0,0,0,1.458615
1188,1.0,2008,70.1140,10.24,94,26.2,27026,0,0,1,0,0,0,0,0,3.849083
772,1.0,2002,217.4790,5.93,71,23.7,1087,0,1,0,0,0,0,0,0,0.139762
1336,1.0,2002,106.0975,12.50,64,25.6,19068,0,0,1,0,0,0,0,0,2.435366
429,1.0,2010,129.3125,10.07,94,26.1,10859,0,0,1,0,0,0,0,0,3.664587


Let's have a look at the data types in the training set to confirm that all is in order. We must only have numerical values.  

In [19]:
X_train_fe.dtypes

,0
const,float64
Year,int64
Adult_mortality,float64
Alcohol_consumption,float64
Measles,int64
BMI,float64
GDP_per_capita,int64
Region_Asia,int64
Region_Central America and Caribbean,int64
Region_European Union,int64


Let's also confirm that we have no null values in our training set.

In [20]:
X_train_fe.isnull().sum()

,0
const,0
Year,0
Adult_mortality,0
Alcohol_consumption,0
Measles,0
BMI,0
GDP_per_capita,0
Region_Asia,0
Region_Central America and Caribbean,0
Region_European Union,0


### 4. Data Scaling

In the process of building the linear model, we observed that training it with features with different ranges can cause imbalances in the model, affecting its performance.\
Let’s examine the range between the maximum and minimum values across all columns in `X_train_fe`.

In [21]:
# This function outputs the value ranges of each feature

def range_checker(df):
    columns = list(df.columns)  # create a list with all the columns in the dataframe
    for i in columns:   # iterate through the list of columns
        print(f'The range of "{i}" column is {df[i].max() - df[i].min()}')  # print the difference between maximum and minimum values

In [22]:
range_checker(X_train_fe)

The range of "const" column is 0.0
The range of "Year" column is 15
The range of "Adult_mortality" column is 669.3625
The range of "Alcohol_consumption" column is 17.75
The range of "Measles" column is 87
The range of "BMI" column is 12.3
The range of "GDP_per_capita" column is 112270
The range of "Region_Asia" column is 1
The range of "Region_Central America and Caribbean" column is 1
The range of "Region_European Union" column is 1
The range of "Region_Middle East" column is 1
The range of "Region_North America" column is 1
The range of "Region_Oceania" column is 1
The range of "Region_Rest of Europe" column is 1
The range of "Region_South America" column is 1
The range of "log_population" column is 7.153500731319662


Let's focus on the non-binary columns.\
We can observe high value ranges in columns such as `GDP_per_capita`, `Population_mln` and `Adult_mortality`.\
On the other hand, there are low value ranges in columns such as `Alcohol_consumption`, `Hepatitis_B`, `Measles`, `BMI` and `Schooling`. Training a linear model with these columns as they are (with such different ranges) would cause imbalances in the model.
###### Note: It was observed that `Incidents_HIV` has an high value range; however, after testing, we concluded that scaling it would increase the condition number of the linear model.

To address this, let's apply scaling to normalize the feature ranges, ensuring model stability and performance.\
We will use `RobustScaler`, which is ideal for data with outliers non-Gaussian distributions, as is the case with this dataset.

In [23]:
# Creating a list of columns to scale
columns_to_scale = ['Year', 'Adult_mortality', 'Alcohol_consumption','Measles','BMI','GDP_per_capita']

# Initialize scaler
scaler = RobustScaler()

# Fit on train and transform both sets
X_train_fe[columns_to_scale] = scaler.fit_transform(X_train_fe[columns_to_scale])
X_test_fe[columns_to_scale] = scaler.transform(X_test_fe[columns_to_scale])

Checking out the first few rows of the scaled dataframe `X_train_fe`.

In [24]:
pd.set_option('display.max_columns', None)
X_train_fe.head(3)

,const,Year,Adult_mortality,Alcohol_consumption,Measles,BMI,GDP_per_capita,Region_Asia,Region_Central America and Caribbean,Region_European Union,Region_Middle East,Region_North America,Region_Oceania,Region_Rest of Europe,Region_South America,log_population
2291,1.0,-0.666667,-0.488750,-0.358006,0.310345,0.50000,4.836784,0,0,0,1,0,0,0,0,1.458615
1188,1.0,0.111111,-0.670939,0.944109,0.379310,0.18750,2.035070,0,0,1,0,0,0,0,0,3.849083
772,1.0,-0.555556,0.382784,0.293051,-0.413793,-0.59375,-0.279672,0,1,0,0,0,0,0,0,0.139762


Running again the `range_checker()` function, we can confirm that the value ranges of the features are more homogeneous.

In [25]:
range_checker(X_train_fe)

The range of "const" column is 0.0
The range of "Year" column is 1.6666666666666665
The range of "Adult_mortality" column is 4.786228988911473
The range of "Alcohol_consumption" column is 2.6812688821752264
The range of "Measles" column is 3.0
The range of "BMI" column is 3.843750000000001
The range of "GDP_per_capita" column is 10.01873996073532
The range of "Region_Asia" column is 1
The range of "Region_Central America and Caribbean" column is 1
The range of "Region_European Union" column is 1
The range of "Region_Middle East" column is 1
The range of "Region_North America" column is 1
The range of "Region_Oceania" column is 1
The range of "Region_Rest of Europe" column is 1
The range of "Region_South America" column is 1
The range of "log_population" column is 7.153500731319662


Now our data is ready for modelling!

### 5. Multicollinearity Assessment

With the `Variance Inflation Factor (VIF)` we can detect multicollinearity in the data. We'll want to avoid multicollinearity as it increases the model's condition number and negatively impacts the robustness and stability of our model.

In [26]:
vif_data = pd.DataFrame()  # create a new dataframe
vif_data['Feature'] = X_train_fe.columns   # create a 'Feature' column which will contain the names of the features in X_train_fe
vif_data['VIF'] = [variance_inflation_factor(X_train_fe.values, i) for i in range(X_train_fe.shape[1])]  # using a loop, calculate the VIF for each feature (list comprehension taken from DataCamp website)
print(vif_data)

                                 Feature        VIF
0                                  const  12.330001
1                                   Year   1.098759
2                        Adult_mortality   2.760844
3                    Alcohol_consumption   2.815632
4                                Measles   1.486231
5                                    BMI   2.504089
6                         GDP_per_capita   1.784864
7                            Region_Asia   1.869708
8   Region_Central America and Caribbean   2.003854
9                  Region_European Union   3.761378
10                    Region_Middle East   2.096451
11                  Region_North America   1.412443
12                        Region_Oceania   1.761421
13                 Region_Rest of Europe   2.046601
14                  Region_South America   1.748529
15                        log_population   1.468079


* Features with **VIF = 1** (no multicollinearity): `Diphtheria_Polio`, `Thinness_combined`
* Features with **VIF between 1 and 5** (moderate multicollinearity): `Hepatitis_B`, `Measles`, `Population_mln`, `Alcohol_consumption`, `BMI`, `Incidents_HIV`, `GDP_per_capita`, `Schooling`, `Region_North America`, `Region_Oceania`, `Region_South America`, `Region_Asia`, `Region_Central America and Caribbean`, `Region_Middle East`, `Region_Rest of Europe`, `Under_five_Infant_deaths`
* Features with **VIF > 5** (high multicollinearity): `Adult_mortality`, `Economy_status_Developing`, `Region_European Union`

There are no features with a **VIF > 10** (serious muticollinearity).

### 6. Modelling

Let's now build an OLS (Ordinary Least Squares) model!

In [27]:
lin_reg = sm.OLS(y_train, X_train_fe) # initialising the model object with 'y_train' and 'X_train_fe'
results = lin_reg.fit() # we fit the model and store it inside results
results.summary() # check out the summary of our model

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:        Life_expectancy   R-squared:                       0.948
Model:                            OLS   Adj. R-squared:                  0.948
Method:                 Least Squares   F-statistic:                     2780.
Date:                Tue, 27 May 2025   Prob (F-statistic):               0.00
Time:                        11:21:20   Log-Likelihood:                -4997.2
No. Observations:                2291   AIC:                         1.003e+04
Df Residuals:                    2275   BIC:                         1.012e+04
Df Model:                          15                                         
Covariance Type:            nonrobust                                         
========================================================================================================
                                           coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------
const                                   68.1587      0.158    431.995      0.000      67.849      68.468
Year                                     0.6107      0.091      6.705      0.000       0.432       0.789
Adult_mortality                         -8.3090      0.091    -91.692      0.000      -8.487      -8.131
Alcohol_consumption                      1.3019      0.125     10.403      0.000       1.057       1.547
Measles                                  0.8358      0.085      9.805      0.000       0.669       1.003
BMI                                      1.3920      0.104     13.374      0.000       1.188       1.596
GDP_per_capita                           0.5586      0.040     14.031      0.000       0.481       0.637
Region_Asia                              2.2544      0.171     13.201      0.000       1.920       2.589
Region_Central America and Caribbean     3.1479      0.208     15.156      0.000       2.741       3.555
Region_European Union                    2.4880      0.245     10.137      0.000       2.007       2.969
Region_Middle East                       1.2240      0.240      5.101      0.000       0.753       1.695
Region_North America                     1.7931      0.418      4.288      0.000       0.973       2.613
Region_Oceania                           0.6800      0.249      2.731      0.006       0.192       1.168
Region_Rest of Europe                    2.4039      0.232     10.363      0.000       1.949       2.859
Region_South America                     2.9734      0.234     12.681      0.000       2.514       3.433
log_population                           0.2650      0.038      7.065      0.000       0.191       0.339
==============================================================================
Omnibus:                       65.978   Durbin-Watson:                   1.931
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              150.431
Skew:                          -0.115   Prob(JB):                     2.16e-33
Kurtosis:                       4.234   Cond. No.                         34.3
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Let's inspect results and focus on R-squared, Adj. R-squared, BIC, AIC, p-value and Cond. No:
* **R-squared**: the model fits the data really good, explaining 96% of the variance.
* **AIC and BIC**: values are around 9000
* **p-values**: generally good (below 0.05), except for the following features: `Measles`, `Population_mln`, `Region_European Union`, `Region_Oceania`, `Thinness_combined`.
* **Cond. No.**: really good, considering that the ideal value is below 300. This indicates that the model is stable and can be trusted.

### 7. Model Evaluation

Let's use the model to predict on the training set `X_train_fe` and calculate the RMSE value.

In [28]:
# Create our prediction results and store them
y_pred = results.predict(X_train_fe)

# Calculate the RMSE of our model on the training set
rmse_train = statsmodels.tools.eval_measures.rmse(y_train, y_pred)

# Print out the result
print(rmse_train)

2.143148295852056


And now we predict on the test set `X_test_fe` and calculate the RMSE value.

In [29]:
# Predict once again using our results
y_test_pred = results.predict(X_test_fe)

# Calculate the RMSE for Test
rmse_test = statsmodels.tools.eval_measures.rmse(y_test, y_test_pred)

# Print out the result
print(rmse_test)

2.1571877542492572


The RMSE values for the training and test sets are considerably close, indicating that the model generalizes well to unseen data and does not exhibit significant overfitting or underfitting.

Let's visualise the predictions against the actual values.

In [30]:
# Create a new DataFrame that holds the actual values (y_test) and the predicted values (y_test_pred)
actual_predicted_df = pd.DataFrame({'Actual': y_test, 'Predicted': y_test_pred})

# Check out first few rows
actual_predicted_df.head()

,Actual,Predicted
473,73.2,73.388059
1178,73.6,74.120563
1135,79.2,77.944591
1549,75.0,72.308203
2626,70.4,69.875552


**----VISUALISATION TO GO HERE----**

### 8. Conclusions

. . .

***